In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torchsummary import summary
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.utils import clip_grad_norm_
import torch_tensorrt

import numpy as np
import gzip
import pickle
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gc
import time
import random
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, accuracy_score, cohen_kappa_score
from collections import deque
import pandas as pd
import math


import sys
sys.path.append('..')
from slp_package.slp_functions import create_merged_game_data_df
from slp_package.input_dataset import InputDataSet
import slp_package.pytorch_functions as slp_pytorch_functions

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using CUDA
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)
torch.cuda.is_available()

True

In [2]:
# We classify 5 characters on competitive stages

source_data = ['public','ranked','mango']

general_features = {
    'stage_name': ['FOUNTAIN_OF_DREAMS','FINAL_DESTINATION','BATTLEFIELD','YOSHIS_STORY','POKEMON_STADIUM','DREAMLAND'],
    'num_players': [2],
    'conclusive': [True]
}
player_features = {
    # 'netplay_code': ['MANG#0'],
    # 'character_name': ['FOX', 'FALCO', 'MARTH', 'CAPTAIN_FALCON', 'SHEIK'],
    # 'character_name': ['PIKACHU','PICHU'],
    # 'character_name': ['FOX', 'CAPTAIN_FALCON', 'SHEIK', 'FALCO', 'GAME_AND_WATCH', 'MARTH', 'LINK', 'ICE_CLIMBERS', 'SAMUS', 'GANONDORF', 'BOWSER', 'MEWTWO', 'YOSHI', 'PIKACHU', 'JIGGLYPUFF', 'NESS', 'DR_MARIO', 'PEACH', 'LUIGI', 'DONKEY_KONG'],
    'character_name': ['FOX', 'CAPTAIN_FALCON', 'SHEIK', 'FALCO', 'GAME_AND_WATCH', 'MARTH', 'LINK', 'ICE_CLIMBERS', 'SAMUS', 'GANONDORF', 'BOWSER', 'MEWTWO', 'YOSHI', 'PIKACHU', 'JIGGLYPUFF', 'NESS', 'DR_MARIO', 'MARIO', 'PEACH', 'ROY', 'LUIGI', 'YOUNG_LINK', 'DONKEY_KONG', 'PICHU', 'KIRBY'], # No ZELDA
    'type_name': ['HUMAN']
    
}
opposing_player_features = {
    # 'character_name': ['MARTH'],
    # 'netplay_code': ['KOD#0', 'ZAIN#0']
    'type_name': ['HUMAN']
}
label_info = {
    'source': ['player'], # Can be 'general', 'player
    # 'feature': ['netplay_code']
    'feature': ['character_name']
}
    

In [3]:
class MappedDataset(Dataset):
    def __init__(self, file_path, dtype, shape):
        self.data = np.memmap(file_path, dtype=dtype, mode='r', shape=shape)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

def prepare_data_loaders(batch_size, num_workers):
    # Initialize datasets
    save_path = "/workspace/melee_project_data/MiniRocket_Transform/"
    
    # Train dataset
    filename = 'minirocket_all_characters_60s_5000_segments_per_character_train.dat'
    full_path = os.path.join(save_path, filename)
    dtype = 'float32'  # Example dtype, adjust as necessary
    shape = (5000, 60, 2048)  # Example shape, adjust as necessary
    train_dataset = MappedDataset(full_path, dtype, shape)
    
    # Test dataset
    filename = 'minirocket_all_characters_60s_5000_segments_per_character_test.dat'
    full_path = os.path.join(save_path, filename)
    test_dataset = MappedDataset(full_path, dtype, shape)

    # Initialize data loaders
    loaders = {
        'train': DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True, pin_memory=True, persistent_workers=True, drop_last=True),
        'test': DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False, pin_memory=True, persistent_workers=True, drop_last=True)
    }
    return loaders

def init(layer):
    if isinstance(layer, nn.Linear):
        nn.init.constant_(layer.weight.data, 0)
        nn.init.constant_(layer.bias.data, 0)



In [4]:
dataset = InputDataSet(source_data, general_features, player_features, opposing_player_features, label_info)

print(dataset.dataset['labels'].value_counts())
# print(list(dataset.dataset['labels'].unique()))
# dataset.dataset.head()

/workspace/slp_jaspar/rocket_classifier_all_data/../slp_package/input_dataset.py:95: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  processed_df = pd.concat([player_1_df, player_2_df], ignore_index=True)


FOX               103069
FALCO              90719
MARTH              53728
CAPTAIN_FALCON     38006
SHEIK              27623
PEACH              17438
JIGGLYPUFF         16374
SAMUS               9524
ICE_CLIMBERS        6849
GANONDORF           6655
YOSHI               5725
LUIGI               5230
DR_MARIO            4202
PIKACHU             4096
LINK                2502
NESS                2306
DONKEY_KONG         2026
GAME_AND_WATCH      1967
MEWTWO              1775
MARIO               1713
YOUNG_LINK          1447
ROY                 1272
BOWSER               940
KIRBY                556
PICHU                230
Name: labels, dtype: int64


In [5]:
labels_order =  dataset.number_of_segments_per_game(3600,5000)
print(labels_order)
labels_order = labels_order['Label'].values

             Label   Count   Shift
0              FOX  102551  120601
1            FALCO   90263  103954
2            MARTH   53538   68557
3   CAPTAIN_FALCON   37820   43246
4            SHEIK   27536   39604
5            PEACH   17367   27064
6       JIGGLYPUFF   16214   24015
7            SAMUS    9489   16294
8     ICE_CLIMBERS    6820   10776
9        GANONDORF    6611    8103
10           YOSHI    5704    8177
11           LUIGI    5210    7765
12        DR_MARIO    4177    6091
13         PIKACHU    4067    6097
14            LINK    2489    3829
15            NESS    2291    4183
16     DONKEY_KONG    2009    2903
17  GAME_AND_WATCH    1949    2305
18          MEWTWO    1758    3257
19           MARIO    1710    2612
20      YOUNG_LINK    1430    2256
21             ROY    1262    1787
22          BOWSER     934    1532
23           KIRBY     531     851
24           PICHU     227     330


In [15]:
# _, _, y_train, y_test  = dataset.train_test_split_numpy(test_ratio = .20, val = False)
# save_path = "/workspace/melee_project_data/MiniRocket_Transform/"

# with gzip.open(save_path + 'minirocket_all_characters_60s_5000_segments_per_character_y_test', 'wb') as f:
#     np.save(f, y_test)
# with gzip.open(save_path + 'minirocket_all_characters_60s_5000_segments_per_character_y_train', 'wb') as f:
#     np.save(f, y_train)


100%|██████████| 80817/80817 [00:00<00:00, 146935.43it/s]


In [16]:
# print(y_train)

['FALCO' 'FALCO' 'FALCO' ... 'SAMUS' 'SAMUS' 'SAMUS']


In [7]:
# import numpy as np
# import os
# import gzip
# from torch.utils.data import Dataset, DataLoader
# from torch import nn

# class MappedDataset(Dataset):
#     def __init__(self, data_path, label_path, dtype, shape):
#         # Memory-mapped data for features
#         self.data = np.memmap(data_path, dtype=dtype, mode='r', shape=shape)
#         # Load labels from compressed file
#         with gzip.open(label_path, 'rb') as f:
#             self.labels = np.load(f)

#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, idx):
#         # Return both the feature and label for the given index
#         return self.data[idx], self.labels[idx]

# def prepare_data_loaders(batch_size, num_workers):
#     save_path = "/workspace/melee_project_data/MiniRocket_Transform/"
    
#     # Train dataset
#     train_data_path = os.path.join(save_path, 'minirocket_all_characters_60s_5000_segments_per_character_train.dat')
#     train_label_path = os.path.join(save_path, 'minirocket_all_characters_60s_5000_segments_per_character_y_train')
#     dtype = 'float32'  # Example dtype, adjust as necessary
#     shape = (5000, 60, 2048)  # Example shape, adjust as necessary
#     train_dataset = MappedDataset(train_data_path, train_label_path, dtype, shape)
    
#     # Test dataset
#     test_data_path = os.path.join(save_path, 'minirocket_all_characters_60s_5000_segments_per_character_test.dat')
#     test_label_path = os.path.join(save_path, 'minirocket_all_characters_60s_5000_segments_per_character_y_test')
#     test_dataset = MappedDataset(test_data_path, test_label_path, dtype, shape)

#     # Initialize data loaders
#     loaders = {
#         'train': DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True, pin_memory=True, persistent_workers=True, drop_last=True),
#         'test': DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False, pin_memory=True, persistent_workers=True, drop_last=True)
#     }
#     return loaders

# def init(layer):
#     if isinstance(layer, nn.Linear):
#         nn.init.constant_(layer.weight.data, 0)
#         nn.init.constant_(layer.bias.data, 0)




In [24]:
# Function to load and encode labels
def load_and_encode_labels(label_path):
    with gzip.open(label_path, 'rb') as f:
        labels = np.load(f)
    return labels

# Prepare and encode labels before initializing datasets
def prepare_encoded_labels(train_label_path, test_label_path):
    # Load labels
    y_train = load_and_encode_labels(train_label_path)
    y_test = load_and_encode_labels(test_label_path)
    
    # Create a label encoder and fit it to all possible labels (train + test)
    encoder = LabelEncoder()
    encoder.fit(np.concatenate((y_train, y_test), axis=0))
    
    # Transform labels to encoded version
    encoded_y_train = encoder.transform(y_train)
    encoded_y_test = encoder.transform(y_test)
    
    return encoded_y_train, encoded_y_test

class MappedDataset(Dataset):
    def __init__(self, data_path, labels, shape):
        # Memory-mapped data for features
        self.data = np.memmap(data_path, dtype='float32', mode='r', shape=shape)
        self.labels = labels

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Return both the feature and label for the given index
        return self.data[idx], self.labels[idx]

def prepare_data_loaders(batch_size, num_workers):
    save_path = "/workspace/melee_project_data/MiniRocket_Transform/"
    
    # Paths
    train_data_path = os.path.join(save_path, 'minirocket_all_characters_60s_5000_segments_per_character_train.dat')
    train_label_path = os.path.join(save_path, 'minirocket_all_characters_60s_5000_segments_per_character_y_train')
    test_data_path = os.path.join(save_path, 'minirocket_all_characters_60s_5000_segments_per_character_test.dat')
    test_label_path = os.path.join(save_path, 'minirocket_all_characters_60s_5000_segments_per_character_y_test')
    
    # Encode labels
    encoded_y_train, encoded_y_test = prepare_encoded_labels(train_label_path, test_label_path)
    
    # Datasets with updated shapes
    train_dataset = MappedDataset(train_data_path, encoded_y_train, shape=(100000, 127932))
    test_dataset = MappedDataset(test_data_path, encoded_y_test, shape=(25000, 127932))

    # Initialize data loaders
    loaders = {
        'train': DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True, pin_memory=True, persistent_workers=True, drop_last=True),
        'test': DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False, pin_memory=True, persistent_workers=True, drop_last=True)
    }
    return loaders

# Example of initializing and using the data loaders
loaders = prepare_data_loaders(batch_size=32, num_workers=16)

In [25]:
# def train_model(model, loaders, loss_function, optimizer, scheduler, num_epochs=10):
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model = model.to(device)
    
#     for epoch in range(num_epochs):
#         model.train()  # Set model to training mode
#         running_loss = 0.0
        
#         # Initialize tqdm with the desired output format
#         with tqdm(loaders['train'], desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as tbar:
#             for inputs, labels in tbar:
#                 inputs, labels = inputs.to(device), labels.to(device)
                
#                 optimizer.zero_grad()  # Zero the parameter gradients
#                 outputs = model(inputs)  # Forward pass
#                 loss = loss_function(outputs, labels)  # Compute loss
#                 loss.backward()  # Backpropagation
#                 optimizer.step()  # Optimize the model

#                 running_loss += loss.item() * inputs.size(0)  # Update total loss

#                 # Set the post description of tqdm for showing batches per second
#                 tbar.set_postfix(loss=(running_loss / (tbar.n + 1)), bps=f'{1/tbar.avg:.2f}')

#         epoch_loss = running_loss / len(loaders['train'].dataset)
#         print(f'Training Loss: {epoch_loss:.4f}')

#         # Validation phase (if you have a validation loader)
#         model.eval()  # Set model to evaluate mode
#         with torch.no_grad(), tqdm(loaders['test'], desc='Validation', unit='batch') as vbar:
#             valid_loss = 0.0
#             for inputs, labels in vbar:
#                 inputs, labels = inputs.to(device), labels.to(device)
#                 outputs = model(inputs)
#                 loss = loss_function(outputs, labels)
#                 valid_loss += loss.item() * inputs.size(0)

#                 # Set the post description of tqdm for showing batches per second in validation
#                 vbar.set_postfix(bps=f'{1/vbar.avg:.2f}')

#             valid_loss /= len(loaders['test'].dataset)
#             print(f'Validation Loss: {valid_loss:.4f}')
        
#         scheduler.step(valid_loss)  # Adjust learning rate based on validation loss


In [31]:
def train_model(model, loaders, loss_function, optimizer, scheduler, num_epochs=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        
        with tqdm(loaders['train'], desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as tbar:
            for data in tbar:
                inputs, labels = data
                # Convert inputs and labels to PyTorch tensors and move them to the device
                inputs = torch.tensor(inputs, dtype=torch.float32).to(device)
                labels = torch.tensor(labels, dtype=torch.long).to(device)
                # print(inputs.shape)
                
                optimizer.zero_grad()  # Zero the parameter gradients
                outputs = model(inputs)  # Forward pass
                loss = loss_function(outputs, labels)  # Compute loss
                loss.backward()  # Backpropagation
                optimizer.step()  # Optimize the model

                running_loss += loss.item() * inputs.size(0)  # Update total loss
                
                # Update the progress bar
                tbar.set_postfix(loss=(running_loss / (tbar.n + 1)))

        epoch_loss = running_loss / len(loaders['train'].dataset)
        print(f'Training Loss: {epoch_loss:.4f}')

        # Validation phase
        model.eval()  # Set model to evaluate mode
        with torch.no_grad(), tqdm(loaders['test'], desc='Validation', unit='batch') as vbar:
            valid_loss = 0.0
            for data in vbar:
                inputs, labels = data
                inputs = torch.tensor(inputs, dtype=torch.float32).to(device)
                labels = torch.tensor(labels, dtype=torch.long).to(device)
                
                outputs = model(inputs)
                loss = loss_function(outputs, labels)
                valid_loss += loss.item() * inputs.size(0)

                # Update the validation progress bar
                vbar.set_postfix(loss=(valid_loss / (vbar.n + 1)))

            valid_loss /= len(loaders['test'].dataset)
            print(f'Validation Loss: {valid_loss:.4f}')
        
        scheduler.step(valid_loss)  # Adjust learning rate based on validation loss




In [32]:
model = nn.Sequential(nn.Linear(127932, 25))
loss_function = nn.CrossEntropyLoss()
loaders = prepare_data_loaders(batch_size=32, num_workers=16)
optimizer = Adam(model.parameters(), lr = .0001)
scheduler = ReduceLROnPlateau(optimizer, factor = 0.5, min_lr = 1e-8, patience = 5)
model.apply(init)
model.to('cuda')

Sequential(
  (0): Linear(in_features=127932, out_features=25, bias=True)
)

In [34]:
train_model(model, loaders, loss_function, optimizer, scheduler, num_epochs=10)

Epoch 1/10:   0%|          | 0/3125 [00:00<?, ?batch/s]/tmp/ipykernel_53056/3092518281.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs, dtype=torch.float32).to(device)
/tmp/ipykernel_53056/3092518281.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long).to(device)
Epoch 1/10: 100%|██████████| 3125/3125 [00:15<00:00, 208.26batch/s, loss=117]


Training Loss: 3.6545


Validation:   0%|          | 0/781 [00:00<?, ?batch/s]/tmp/ipykernel_53056/3092518281.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs, dtype=torch.float32).to(device)
/tmp/ipykernel_53056/3092518281.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long).to(device)
Validation: 100%|██████████| 781/781 [00:03<00:00, 233.31batch/s, loss=246]   


Validation Loss: 7.3763


Epoch 2/10: 100%|██████████| 3125/3125 [00:15<00:00, 205.90batch/s, loss=117]


...

Training Loss: 3.5836


Validation: 100%|██████████| 781/781 [00:03<00:00, 230.57batch/s, loss=280]   


Validation Loss: 8.5772


Epoch 5/10:  29%|██▊       | 891/3125 [00:04<00:10, 204.45batch/s, loss=115]  


KeyboardInterrupt: 